In [ ]:
import os
# Setting directory to main course project work
os.chdir('/Users/nicolobrunello/Desktop/RecommenderSystems/RecSysChallenge2021/RecSys_Course_AT_PoliMi')


import os
import numpy
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Utils.load_URM import load_URM 
from Utils.load_ICM import load_ICM 
from scipy import sparse as sps
import pandas as pd
import scipy.sparse as sps

from Data_manager.ChallengeDataset.ChallengeDataset import ChallengeDataset


In [ ]:
def combine(ICM: sps.csr_matrix, URM : sps.csr_matrix):
    return sps.hstack((URM.T, ICM), format='csr')


dataset_object = ChallengeDataset()
dataset = dataset_object.load_data()
URM_train, URM_test = split_train_in_two_percentage_global_sample(dataset.get_URM_all(), train_percentage = 0.80)


#URM_all = load_URM('Data/data_train.csv')
print(URM_train.shape)
ICM_channel_all = load_ICM('Data/data_ICM_channel.csv')
ICM_event_all = load_ICM('Data/data_ICM_event.csv')
ICM_genre_all = load_ICM('Data/data_ICM_genre.csv')
ICM_subgenre_all = load_ICM('Data/data_ICM_subgenre.csv')
ICM_all = sps.hstack((ICM_channel_all, ICM_event_all, ICM_genre_all, ICM_subgenre_all))
print(ICM_all.shape)
URM_ICM_all = combine(ICM=ICM_all, URM=URM_train)

print(URM_ICM_all.shape)

In [ ]:
print(URM_ICM_all.T.shape)

In [ ]:
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_ICM_all, verbose=True)
# Starting hyperparameters from Sanvito repo
''' PARAMETRI DI SANVITO STANDARD
SLIM_recommender.fit(
        alpha=0.00026894910579512645,
        l1_ratio=0.08074126876487486,
        topK=int(395.376118479588),
        workers=6
    )
'''

#PARAMETRI CON CUI HO FATTO LA SUBMISSION DA 0.47699
recommender.fit(
        alpha=0.00027153127362632663,
        l1_ratio=0.07989965746223235,
        topK=int(397.147437437882),
        workers=6
    )


'''recommender.fit(
        alpha=0.007596772625600448,
        l1_ratio=0.09354638545594608,
        topK=410,
        workers=6
    )'''

In [ ]:

from Recommenders.Recommender_import_list import *
import scipy.sparse as sps

from Data_manager.ChallengeDataset.ChallengeDataset import ChallengeDataset
from Data_manager.DataSplitter_leave_k_out import DataSplitter_leave_k_out
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping
from Recommenders.BaseCBFRecommender import BaseItemCBFRecommender, BaseUserCBFRecommender
from Evaluation.Evaluator import EvaluatorHoldout
import traceback, os

evaluator = EvaluatorHoldout(URM_test, [5, 10], exclude_seen=True)
results_run_1, results_run_string_1 = evaluator.evaluateRecommender(recommender)
output_root_path = "./result_experiments/"
logFile = open(output_root_path + "result_S-Slim.txt", "a")

recommender.save_model(output_root_path, file_name = "temp_SSlim.zip")
print("Algorithm: S-Slim, results: \n{}".format( results_run_string_1))
logFile.write("Algorithm: S-Slim, results: \n{}\n".format(results_run_string_1))
logFile.flush()
